In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_ones_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/central_pixel_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip


In [4]:

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import zipfile

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [6]:
data = pd.read_csv("../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip", compression="zip")

In [7]:
data["GalaxyID"] = data["GalaxyID"].apply(lambda x: f"{x}.jpg")

In [8]:
filtered = data[(data["Class1.1"] >= 0.8) | (data["Class1.2"] >= 0.8)].drop_duplicates().reset_index(drop=True)

In [9]:
conditions = [filtered["Class1.1"] >= 0.8, filtered["Class1.2"] >= 0.8]
choices = ["elliptical", "spiral"]
filtered["class"] = np.select(conditions, choices, default="Tie")


In [10]:
df_spiral = filtered[filtered["class"] == "spiral"].sample(n=8000, random_state=42)
df_elliptical = filtered[filtered["class"] == "elliptical"].sample(n=8000, random_state=42)


In [11]:
df_galaxies = pd.concat([df_spiral, df_elliptical])[["GalaxyID", "class"]]
le = LabelEncoder()
df_galaxies["label"] = le.fit_transform(df_galaxies["class"])  # 0 = elliptical, 1 = spiral


In [12]:
df_galaxies.head()

,GalaxyID,class,label
20666,863568.jpg,spiral,1
17422,745024.jpg,spiral,1
23878,985580.jpg,spiral,1
15765,684719.jpg,spiral,1
6305,340999.jpg,spiral,1


In [13]:


class GalaxyBinaryDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["GalaxyID"])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["label"]).long()
        if self.transform:
            image = self.transform(image)
        return image, label


In [14]:


train_df, test_df = train_test_split(
    df_galaxies, test_size=0.2, random_state=42, stratify=df_galaxies["label"]
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = GalaxyBinaryDataset(train_df, "./images_training_rev1", transform)
test_dataset = GalaxyBinaryDataset(test_df, "./images_training_rev1", transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [15]:


model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 233MB/s]


In [16]:


def train(model, loader):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# Train for a few epochs
for epoch in range(5):
    loss = train(model, train_loader)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


Epoch 1, Loss: 0.1096
Epoch 2, Loss: 0.0421
Epoch 3, Loss: 0.0257
Epoch 4, Loss: 0.0261
Epoch 5, Loss: 0.0200


In [18]:

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"\n Test Accuracy: {acc:.4f}")



 Test Accuracy: 0.9769
